# 인스타그램
작성 날짜, 내용 출력

In [1]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import os
import urllib.request
import warnings # 경고메세지 제거
import requests
from datetime import datetime, timedelta
import pandas as pd

warnings.filterwarnings(action='ignore') # 경고 메세지 제거

# 로그인 계정 정보 [ID : dawaplease / PWD : dawa20152017]
ig_id = 'dawaplease'
ig_pwd = 'dawa20152017'

# 크롬드라이버 호출
driver = webdriver.Chrome("C:/project/chromedriver.exe")

# 로그인 [ID : dawaplease / PW : dawa20152017]
driver.get('https://www.instagram.com/accounts/login/')

time.sleep(3)

id_input = driver.find_elements_by_css_selector('#loginForm > div > div > div > label > input')[0]

id_input.send_keys(ig_id)
password_input = driver.find_elements_by_css_selector('#loginForm > div > div > div > label > input')[1]
password_input.send_keys(ig_pwd)
password_input.submit()
time.sleep(3)

# 작품 검색
search_key = input("!주의! 검색 명내 띄어쓰기 사용 X !\n작품 검색 #")
print()
search_ig = "https://www.instagram.com/explore/tags/" + search_key + "/"

# 작품 검색 페이지 로드
driver.get(search_ig)
time.sleep(3) #웹 페이지 로드 시간 : 3초

# 마지막 결과까지 스크롤
SCROLL_PAUSE_TIME = 1.0
post_list = []

# 중복 제거
def duplicate(my_list):
    my_set = set(my_list) # 집합으로 변환
    new_list = list(my_set) # 리스트로 변환
    return new_list

# 포스트 링크 추출
while True:
    html = driver.page_source
    soup = BeautifulSoup(html, "lxml")

    for link1 in soup.find_all(name="div",attrs={"class":"Nnq7C weEfm"}):
        for num in range(3):
            try:
                title = link1.select('a')[num]
                real = title.attrs['href']
                post_list.append(real)
            except:
                break

    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(SCROLL_PAUSE_TIME)
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")
        
        if new_height == last_height:
            break
            
        else:
            last_height = new_height
            continue

post_list = duplicate(post_list) # 중복포스트 제거
post_num = len(post_list) # 검색 결과 개수
print("총 "+str(post_num)+"개의 데이터.\n")

# 포스트 데이터 추출
csvtext = []

for i in range(0, post_num):
    csvtext.append([])
    post_url = 'https://www.instagram.com'+ post_list[i]
    driver.get(post_url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')

    # 내용
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ''
    csvtext[i].append(content)
    print(content)
    
    # 작성날짜
    try:
        date = soup.select('time._1o9PC.Nzb55')[0]['datetime'][:10]
    except:
        date = ''
    csvtext[i].append(date)
    print(date)

# DataFrame
print(str(i+1)+"개의 데이터 받아오는 중.\n")
data = pd.DataFrame(csvtext, columns = ['내용', '작성 날짜'])
print("저장 완료.\n")

driver.close() # 크롬 드라이버 종료

!주의! 검색 명내 띄어쓰기 사용 X !
작품 검색 #내가키운s급들

총 969개의 데이터.

#내가키운s급들 #근데 둘만좋아하네#아빠껀홍삼#내껀 손톱보호제~~#어버이날승아가 준비하고 좋아서 내게 달려온이유 아빠가 사랑한다는 말을 말로도아닌 문자로 처음 받았다고~~^^알고있었지만 ᆢ이렇게해주니  평생캡쳐해서 갖고있겠다고 처음이라고 마음은 알고있지만 표현해주니 더 열배로 좋아해주는 승아를 보니 표현의 중요성에대해나도 ~~한번 ready go!!!!!
2020-05-08
#내스급 #한유진 #팬아트 #내가키운s급들 #일러스트 #그림 요즘 보는 웹소설 팬아트. 일 그림 이외에 팬아트는 정말 오랜만에 그리는거라 그리면서 재미있었다. 항상 여자만 그리다가 남자 그리니까 재미있어!
2020-03-05
[19.11.23]내가키운S급들_성현제[개인샷]Cos-쁠Photo_정화님세성길드장님 사랑하는만큼 너무 어렵다요#내스급코스프레 #내가키운s급들 #성현제 #목성의_주인_이_세상을_당신에게 #cosplay #selfie #내스급_성현제 #コスプレ #コスプレ好き#촬영 #정장 #쓰리피스수트#개인촬영 #cos #コスプレイア
2019-11-27
목표는 15개! 드뎌 끝이 보인다😎 끝내면 삐약이 만들수있어!! 목 장식은 급한건 아니니까 천천히 사고 표정 끝나면 바로 삐약이 작업 고~~ #내스급 #양인형 #손재주 #인형 #태산이 #송송이 #소설 #취미 #바느질 #내가키운s급들
2019-11-14
귀여운 캐릭터는 당분간 안녕....#1부코 #체인질링 #내스급 #내가키운s급들 #부산코믹월드 #팀코 #내스급팀코 #코스프레 #코스어 #cosplay #costumplayer #コスプレ #Tae_mincos
2020-01-12
내가 키운 S급들보조계 S급 헌터노아 루히르#cosplay #내가키운s급들 #판타지소설 #fantasynovel #노아루히르 #루씨코스
2019-07-31
내가 키운 S급들(내스급)강소영P. @dm_pher#cosplay #cosplayphotography #내가키운s급들 #내스

#illustration #일러스트 #イラスト #내가키운s급들 #내스급
2019-10-26
⠀💙⠀파도랑 물의 파장을 표현하고 싶어서 연출에 공을 많이들였다! 영상도 잘 찍혔고 색이 청록~파랑으로 예쁘게 나와줘서 기쁘다 파랑사랑맨은 행복٩(●'▿'●)۶⠀⠀#캘리그라피 #calligraphy #잉크 #붓펜 #붓펜캘리#내가키운S급들 #박예림 #물의지배자
2020-04-11
🔥집코 20.04.22🔥(2)내가 키운 s급들 - 한유진#내가키운s급들 #한유진 #내가키운s급들코스프레 #내스급 #내스급코스프레 #한유진코스프레 #코스프레 #코스어 #コスプレ #コスプレイヤー #cosplay #cosplayer #kawaiicosplay #cosplaying #コスプレ好き #cosplayphotography #コスプレイ
2020-04-22
#내가키운S급들#박예림#물의지배자cos : 꼬맴 삼락공원에서2020.4.18
2020-04-19
_현제유진 폰스트랩😘-eBook “내가 키운 S급들”의 주인공현제유진의 폰스트랩이에요. 자세히 디자인 보시면 조금씩 다양한캐릭터가 나열되어 있어요🌸-양면 디자인으로 원하시는 쪽으로 선택하여 붙일 수 있게 디자인되었어요! 무선충전, 핸드폰거치대, 셀카봉 이제는 자유롭게 사용하세요. 얇은 두께감의 폰스트랩이기때문에 모두 사용가능합다.-🖥 https://m.blog.naver.com/keyringkeyring1/221859621171#키링키링 #폰스트랩 #스마트루프 #핑거스트랩 #핑거루프 #폰스트랩제작 #주문제작 #단체제작 #행사기념품 #굿즈제작 #폰스트랩주문 #무선충전되는 #핸드폰스트랩 #일상 #인싸템 #폰스트랩디자인 #일러스트그림 #현제유진 #내가키운s급들 #ebook #웹소설 #현제 #유진 #phonestrap #smartloop #keyringkeyring #made #design #illustration
2020-04-09
새롬, 효주, 현학, 수정! 서코 가다 두둥! 가서 눈 호강하고 발의 감각을 잃었다 ㅋㅋ 서코 가서 산건 정작 소설 굿즈 ㅋ 

#내가키운S급들 #내스급 #성현제
2019-06-06
💄💋.....#illustration #illust #illustrator #illustrations #art #artwork #artist #artistsoninstagram #일러스트 #그림 #그림스타그램 #내가키운S급들 #내스급 #doodle #fanart #일러스트레이션 #일러스트레이터 #イラスト #イラストレーション #lipstick #립스틱 #follow #いいね返し
2020-08-05
🎃✨ HALLOWEEN ✨🎃''[ 내가 키운 S급들 ]박예림P :: @cos_pika....#cosplay #cos #코스프레 #コスプレ #コスプレイア #내가키운s급들 #내스급 #내가키운s급들코스프레 #내스급코스프레 #박예림 #박예림헌터 #내스급박예림 #내스급_박예림 #판소 #판타지소설 #할로윈 #halloween #halloweencostume #halloween🎃
2019-11-05
성현제 생신 축전 그렸습니다...⚡️ 근데 리본 잘못그린걸 이제야 알았다 #내가키운s급들 #내스급
2019-08-29
#인형제작 #인형만들기 #내가키운s급들 #내스급 #이린 #이린인형 #불의정령 안에 철사가 너무 짧아서 얼굴을 들다만 ㅠㅠ
2019-09-02
#내스급 #내스급_박예림 #생일 #내가키운s급들 #타이포 #예쁜글씨 #타이포그래피 #타이포그라피 #취미 #커미션 #프로필링크 #작업물 #로고 #로고제작 #로고디자인 #디자인 #로고커미션 #주문제작
2020-04-11
20190601성현제 곰돌이짤 너무 귀여워서 그려버림..인삐가 보이지만 대충 넘깁시다. 프사로 맘대로 쓰세요. 인스타라 저장은 안되겠지만! 일단 내가 먼저 쓸거임#내스급 #내가키운s급들 #성현제 #1일1그림 #프리인장 #freeicon #character #drawing #draw #キャラクター
2019-05-31
오랜만에 유진이#그림 #그림스타그램 #낙서 #낙서스타그램 #한유

주말엔 밤샘이지!#셀스타그램 #일상스타그램 #1호선 #서울까지 #고고씽#내스급 #내가키운s급들 #생일 #광고판#충무로 #충무로맛집 #파스타마켓 #존맛탱#메가박스 #메박 #올나잇
2019-08-04
🐈#그림#그림그리기 #일상 #그림스타그램 #그림#드로잉 #취미미술 #drawing #일러스트 #취미 #art #힐링 #데일리 #artwork #수채화 #미술 #illustration #손그림 #스케치 #그림일기 #취미생활 #낙서#내스급 #웹소설 #내가키운s급들#선팔#맞팔 #맞팔해요
2020-07-01
#내가키운s급들 #내스급 #성현제 #목성의_주인_이_세상을_당신에게 #성현제_생일축하해 #여름의_가장자리_성현제_생일축하해
2019-08-30
내스급 120화까지 봤당노아 완전 댕댕이ㅜㅜㅜㅡㅜㅡ#내가_키운_s급들 #내스급 #내가키운s급들 #팬아트 #내스급_노아 #그림쟁이 #그림스타그램🎨 #그림스타그램 #시험기간😢 #낙서스타그램
2019-11-30
#illustration #drawing #내가키운s급들 #내스급 #한유진 한유진의 세계란...
2019-09-03
#내스급 #내가키운s급들 #성현제 #그림 #일러스트 #드로잉 #art #illust #drawing #イラスト
2020-04-19
지인한테 만들어준 #내가키운s급들 #박예림 테마 귀걸이.
2019-07-26
Wwww#스윗게일호야#구체관절인형#구관#스윗게일#스윗게일 호야 #호야#sweetgale#sweetgalehoya#돌스타그램#bjdphotography #bjddoll #bjd #구관#구체관절인형#인형#doolstagram #dollstagram #dollsta#可愛い#人形#球形#娃娃#球体関節人形#내스급#내스급박예림#내가키운s급들
2020-02-27
한유진 #한유진 #내스급 #내가키운s급들 #내스급_한유진 #그림 #그림스타그램
2020-05-21
소설에서 최애캐 생일이라 트위터 헤더랑 프사 많이 만들음^^((뿌듯)) #한유현_생일축하해#내가키운s급들#한유현#1225
2019-12-24
As you wish #내스

숙제 하려고 하는데 유진이들(?) 때문에 집중이 안된다... #한유진 #내가키운s급들
2020-02-09
내가 키운 S급들 박예림.PHOTO by 포드 (트위터 아이디 @ COS_POD)
2020-08-11
블루레몬에이드 예림💙🍋 .내가키운s급들 에 나오는 캐릭터입니다...#내스급 #내가키운s급들 #ipaddrawing #characterdrawing #characterillustration #illust #painting #digitalart #digitaldrawing #イラスト #그림 #그림그리기 #일러스트 #그림스타그램 #아이패드그림 #bluelemonade🍋
2020-06-20
(ㅇωㅇ)V= 3 =#cosplay #내가키운s급들 #박예림 #물의지배자
2019-11-10
#switchhwayeon #fromswitch #balljointeddoll #bjd #스위치돌 #내스급 #성현제 #내가키운s급들
2019-10-10
#DOODLE #낙서 #illustration #일러스트 #pencildrawing #drawing #연필 #드로잉 #내가키운s급들 #내스급 #성현제
2019-09-25
#내스급 #내가키운s급들 #성현제 #판소 #덕질 #드로잉 #낙서 #일러스트 #그림 #drawing #art #style #colors #아트 #연습 #illustration #artwork
2020-02-11
오랜만에 촬영했다... 그것도 남캐................#내가키운s급들 #내스급 #유현 #한유현 #코스프레 #한유현코스프레
2020-06-05
#코스프레 #코스어 #cosplay #costume #costumeplay #cosplayer #내스급코스프레 #내스급_강소영 #내가키운s급들 #내스급
2019-08-24
#그림 #그림그리기 #그림러 # 그림쟁이 #내스급 #내가키운s급들 #내가_키운_s급들 #한유진 #송태원 #송실장 

...

내스급도 좋은데..#내스급 #내가키운s급들 #한유진 #성현제 #박예림 #헌터 #s급 #한유현#소설
2020-05-28
여름과 해바라기와 푸른 악세서리#일러스트 #수채화 #그림 #그림스타그램 #illustration #내가키운s급들 #내스급박예림 #박예림헌터 #박예림
2019-08-17
#내가키운s급들 #내스급#한유현
2019-06-02
#ipaddrawing #characterdrawing #characterillustration #illust #painting #digitalart #digitalpainting #イラスト #그림 #그림그리기 #아이패드그림 #일러스트 #그림스타그램 #내스급 #내가키운s급들
2020-07-05
내스급 한유진#그림 #그림스타그램 #일러스트 #일러스트그램 #팬아트그램 #팬아트 #내가키운s급들 #내스급 #illust #illustration #fan_art #fanart #paint #painting #painting🎨 #art #artwork #clipstudio
2019-08-18
한유현🖤❤
2020-07-28
마석 먹는 삐약이#내가키운S급들 #삐약이
2019-04-27
🔗⚡당신을 xx합니다.🔗 #한유진 #내가키운s급들 #내스급 #그림 #그림스타그램 #낙서 #draw #drawing
2019-10-16
은근히 손 많이 갔던 귀걸이ㅜㅜ.....그치만 다들 예쁘다고 해주셔서 좋았다!#내가키운s급들 #내스급_박예림 #내스급 #창랑인어여왕_귀걸이 #귀걸이 #수공예 #수공예귀걸이 #2차창작
2020-01-17
[ 내가 키운 S급들 ]박예림💧P. 벤카님....#cosplay #cos #코스프레 #コスプレ #コスプレイア #내가키운s급들 #내스급 #내가키운s급들코스프레 #내스급코스프레 #박예림 #박예림헌터 #내스급박예림 #내스급_박예림 #판소 #판타지소설
2019-08-15
#다이어리 #다꾸 #내스급 #내가키운s급들 #박예림
2020-04-20
⚡️...#그림 #그림스타그램 #그림그리기 #낙서 #낙서스타그램 #일러 #일러스트 #일러스트그램 #자

뿌리깊은 고트가문 집을 뺏었따!!! 성현제 역시 세성 길드장 :) 집 들어가서 난로 불도 켜서 셀카 찰칵📸 이제 신부 유진이만 있으면 완벽하다....캐릭터 뜨는게 ‘성’으로 뜨는 건 불편하다만,,,😂-#심즈#더심즈 #더심즈4 #심즈4 #심즈4cc #심즈4남심 #심즈남심 #심즈cc #심즈스타그램 #심즈4스샷 #심즈스샷 #sims #thesims #thesims4 #sims4 #sims #thesims4cc #simstagram #simslife #thesims4life #thesims4story #내가키운s급들 #내스급 #내스급_성현제 #성현제 cc템 부족으로 쪼꼼 많이 덜 비슷하게 생긴건 조금 아쉽,,, :(
2020-01-19
대구 윗츠 2호점에서“내가 키운 S급들” 노아 루히르cos : 천P.B : 바람처럼특별히 보정할게 없이 맘에 드는 컷사진 찍으로 대구까지 갈 줄이야...2019.12.22#대구윗츠스튜디오#코스어촬영#내가키운s급들#인물촬영#스튜디오촬영#조명촬영#코스어
2019-12-24
내가 키운 S급들 - 박예림p. 델리만쥬님#내스급 #내스급코스프레 #박예림 #내가키운s급들 #내스급_박예림 #cosplay #cosplayer #model
2019-10-31
[내가 키운 S급들 - 노아 루히르]#내가키운s급들 #cosplay #cosplayer
2020-06-14
노아 🐉pb. @sol_jjikjjik#내가키운s급들 #내스급 #내가키운s급들코스프레 #내스급코스프레 #내스급_노아 #내스급_노아_코스프레 #노아루히르 #코스프레 #cosplay #コスプレ
2020-03-11
#내가키운s급들 #뱀파이어 #한유진 #코스프레모델 #코스프레 #cosplay #uni #model 📸줌스튜디오 💕
2020-01-27
유현냥#illustration #illust #characterillust #그림 #캐릭터일러스트 #내가키운s급들 #내스급 #내스급_한유현 #한유현
2020-07-26
#성현제 #판소 #판타지 #내스급 #내가키운s급들 #cospla

내가 키운 S급들(내스급)강소영P. @dm_pher#cosplay #cosplayphotography #내가키운s급들 #내스급 #강소영 #코스프레
2020-06-11
#내가키운S급들 #송태원 #활자남팬분께서 주문해주신 호텔키링입니다! 블랙과 장미파츠가 잘 어우러져서 한층 고급진 느낌을 내는 키링입니다☺️거기에 고급체까지!!!무사히 배송까지 보내드렸습니다!.※같은 제품의 키링은 만들어 드리지않습니다.
2020-02-10
#내가키운S급들 #유은혜 #초커 #미완성
2019-11-04
내스급 한유현 정장/사복 컨셉 촬영#내가키운s급들 #내스급 #내스급_한유현 #한유현 #코스프레 #코스어 #코스튬플레이어 #コスプレイヤー #コスプレ#コスプレイ #cosplay #cosplayer #costumeplayer
2019-08-17
[ 내가 키운 S급들 ]박예림💧P. 벤카님....#cosplay #cos #코스프레 #コスプレ #コスプレイア #내가키운s급들 #내스급 #내가키운s급들코스프레 #내스급코스프레 #박예림 #박예림헌터 #내스급박예림 #내스급_박예림 #판소 #판타지소설
2019-08-20
20200622 - 내가 키운 S급들🐤 성현제 👔#cosplay #내가키운s급들 #웹소설 #コスプレ
2020-08-07
성현제하고 생각한건데 컨셉촬가고 싶다 얼마일까 #cosplay #내스급코스프레
2019-09-14
2020.1.5 / 119회 부산코믹 일요일내가 키운 S급들 - 리에트p.b 백연님 (@photo_baekyeon_ ) #koreancosplayers#cosplayers #cos #cosplay#내가키운s급들 #내가키운s급들코스프레 #내스급 #내스급코스프레 #내스급_리에트 #내스급_리에트_코스프레 #리에트 #리에트루히르 #코스프레 #한국코스프레 #코스프레촬영 #부산코믹월드 #부산코믹월드119회
2020-01-07
작업끝났오🥰 모두 내일 배송간다아뎌스에서 주문 받은 인형도 낼 배송><저렇게 통에 넣어서 보내면 다들 통에서 인형 안꺼내는데 귀여우니까 인정....#내스급 #내가키운

인형만큼 작아진 유현이가 형을 찾고 있네요! 형은 어디 있으려나~....#내스급 #내가키운s급들 #내스급_한유현 #내스급_한유현 #미니미니 #인형 #윻윶 #유현유진 #그림 #연성 #art #SD #쪼꼬미 #하트 #clipstudio #clipstudiopaint
2020-07-11
Commissionㄱㄱㅇ님 커미션 완료#pixelart#draw#drawing#dotart#pixel #portrait #artwork #artstagram #픽셀아트 #illust #illustration#그림#성현제#내스급 #내가키운s급들#SungHyunJe
2019-11-28
::내가 키운 S급들-노아 루히르::#내가키운s급들
2020-04-08
happy yerim day!#hbdpyr #happyyerimday #내가_키운_S급들 #내가키운s급들 #thesclassesthatiraised #parkyerim #bakyerim
2020-04-10
오타쿠의 책상. 이제 성현제 보면서 마감한다~!!!이제 2월 서코 준비해야지~ 그 전에 덕질하면서 연성 제일 맛있는 윻윶을 먹어야지!♡ 현윶도 존맛.#책상정리 #와콤타블렛 #내가키운s급들 #새벽의연화 #윻윶 #현윶
2020-02-01
어디가 이상한지 보이는데 고칠 기력 없어...#팬아트 #fanart #웹소설 #내가키운s급들 #내스급 #한유현 #doodle #sketchbook #illust #illustration #일러스트 #일러스트레이션 #イラスト #イラストレーション #иллюстрация #visualart #art #artwork #그림스타그램 #별킹
2020-01-05
🔥집코 20.04.22🔥(1)내가 키운 s급들 - 한유진#내가키운s급들 #한유진 #내가키운s급들코스프레 #내스급 #내스급코스프레 #한유진코스프레 #코스프레 #코스어 #コスプレ #コスプレイヤー #cosplay #cosplayer #kawaiicosplay #cosplaying #コスプレ好き #cosplayphotography #コスプレ

한유진✨#내스급 #내가키운s급들 #한유진 #피스 #삐약이
2020-01-22
[키스] kissTool : galaxy note7fe medibang paint.쉬는 게 오래갈 거 같아, 그 동안 기다리실 구독자분들을 위한 엽서이벤트를 진행하려고 합니다!(늦은 내스급 300화 돌파기념!!!!!!!21일 오전 문피아기준 301화).. 이 그림은 이번에 서로 그림을 교환하기로 한 친구의 요청으로 그린 그림인데요... 요새 유명한 판타지소설 팬아트 입니다!:) 그럼 여기서 이벤트 퀴즈!위 그림의 모티브가 된 명화를 덧글로 적어주세요! 정답을 맞추신 분들 중 추첨을 통해 10분 께 그림의 엽서를 보내드립니다!.. >>이벤트 종료!!<< ...+배송 문제류 한국 거주자 분들만 가능합니다! 적는 다는 걸 깜빡해서... 해외 거주자 분들은 다음 기회에 ㅠㅠ 정말 죄송합니다ㅠㅠㅠ#내스급 #성현제 #한유진 #내가키운s급들 #이벤트 #판소 #클림트 #kiss #gustavklimt #키스 #fanart #gold#두들 #illustration #illust #doodle #art #artistsoninstagram #그림 #드로잉 #그림스타그램 #인스타그림 #イラスト#artwork #digitalpainting #digitalart #drawing #드로잉 #일러스트 #art_fashion#팬아트
2019-08-22
“ ..유현아. ”#내가키운s급들 #내스급 #내스급_한유진 #한유진 #회귀전한유진
2020-04-25
이때까지만해도 몰랏음... 초상집분위기될줄
2019-08-30
생일파티를 위해 주문한 케이크♡잊고있어서 급하게 주문했지만 저의 무리한 요청사항까지 맞춰주셨답니다!친구가 픽업가서 놀래구 왔대요ㅋㅋ 진짜예뻐요~♡번개같은Z모양 #이니셜케이크 #숫자케이크 #생일케익 #케이크주문제작 #디저트날다 #디저트맛집 #생화케이크 #팔로환영#내스급 #내가키운s급들 #성현제
2019-09-01
20.01.11[내가 키운 s급들]아크의 길드장 노아 루히르입니다. .p.b 뜸애랑님 (@ lk

In [2]:
data

,내용,작성 날짜
0,#내가키운s급들 #근데 둘만좋아하네#아빠껀홍삼#내껀 손톱보호제~~#어버이날승아가 준...,2020-05-08
1,#내스급 #한유진 #팬아트 #내가키운s급들 #일러스트 #그림 요즘 보는 웹소설 팬아...,2020-03-05
2,[19.11.23]내가키운S급들_성현제[개인샷]Cos-쁠Photo_정화님세성길드장님...,2019-11-27
3,목표는 15개! 드뎌 끝이 보인다😎 끝내면 삐약이 만들수있어!! 목 장식은 급한건 ...,2019-11-14
4,귀여운 캐릭터는 당분간 안녕....#1부코 #체인질링 #내스급 #내가키운s급들 #부...,2020-01-12
...,...,...
964,내스급 예림 #그림 #그림러 #그림쟁이 #일러스트 #draw #drawing #il...,2019-09-09
965,내가 키운S급들 박예림#illustration #artwork #art #그림그리기...,2020-06-21
966,[내가키운 S급들]한유진 cos.sioel#내스급 #내스급코스프레 #내스급_한유진 ...,2019-10-14
967,"내가 키운 S급들, 박예림M. 호브 님📸 @ixi.wdeer",2019-12-01


# 트위터
사용패키지 | https://github.com/twintproject/twint | import twint
c = twint.Config() | twint config 선언
c.Limit = 50 | 수집할 트윗의 개수를 한정할 때 사용
c.Search = '전지적 독자 시점' | "전지적 독자 시점"은 정확히 글자 그대로 적힌 글을 수집
c.Since = '2019-01-01' | 트윗 수집 시작일
c.Until = '2020-01-01' | 트윗 수집 마지막일
c.Popular_tweets = True | True | Polular 탭, False | Recent 탭
c.Store_csv = True | 결과 저장시 csv 포맷으로 저장할 것인가
c.Output = "전지적독자시점.csv" | 결과 저장
c.Store_json = True | JSON 포맷으로 저장할 것인가
c.Hide_output = True | True일 때 주피터 노트북 콘솔에 output이 출력되는 옵션을 끔

In [2]:
!pip install twint

  Created wheel for twint: filename=twint-2.1.20-py3-none-any.whl size=33926 sha256=b4b51c698d012b572bf283fb5342c600aa32cb70011631c9f04819ae2f81630b
  Stored in directory: c:\users\shung\appdata\local\pip\cache\wheels\44\fc\77\99887a36b5c265a87516158858697d1a0b8f32c4d4dbddbb24
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13489 sha256=b00894f561181727a05bb185ef964ed3fd37baa470eaf25eff7af8608b288ceb
  Stored in directory: c:\users\shung\appdata\local\pip\cache\wheels\ed\f7\62\50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for googletransx: filename=googletransx-2.4.2-py3-none-any.whl size=15974 sha256=a016035f8d79a36c11e050176dcbeb353e3390811cc0884505644bf3b7699a7e
  Stored in directory: c:\users\shung\appdata\local\pip\cache\wheels\66\d5\b1\31104b338f7fd45aa8f7d22587765db06773b13df48a89735f
Successfully built twint fake-useragent googletransx


Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [7]:
# Import packages
import twint
import pandas as pd
from os import mkdir, path
import datetime
import re

# Valid Directory Naming Function
def clean_name(term):
    cleaned = re.sub('[^0-9a-zA-Zㄱ-ㅎ가-힣]', '', term) # 특수문자 제거
    return cleaned

# Twint Search Function
def twint_search(dirname, searchterm, since, until, csv_name):

    c = twint.Config()
    
    # Parameter setting
    #c.Limit = limit
    c.Search = searchterm
    c.Since = since
    c.Until = until
    c.Hide_output = True
    c.Store_csv = True
    c.Output = csv_name
    c.Debug = True
    c.Resume = f'{dirname}/save_endpoint/save_endpoint_{since}.txt'
    c.Popular_tweets = True
    
    try:
        twint.run.Search(c)

    except (KeyboardInterrupt, SystemExit):
        raise

    except:
        print(f"Problem with {since}.")

# Loop Function
def twint_loop(searchterm, since, until):
    
    dirname = clean_name(searchterm)
    
    # Create target directory
    try:
        mkdir(dirname)
        mkdir(f'{dirname}/save_endpoint')
        print("Directory" , dirname ,  "Created ")
    except FileExistsError:
        print("Directory" , dirname ,  "already exists")
    
    # Loop 
    daterange = pd.date_range(since, until)
    
    for start_date in daterange:

        since = start_date.strftime("%Y-%m-%d")
        until = (start_date + datetime.timedelta(days=1)).strftime("%Y-%m-%d")

        csv_name = "".join(since.split("-")) + ".csv"
        csv_name = path.join(dirname, csv_name)

        print(f'Getting {since} ')
        twint_search(dirname, searchterm, since, until, csv_name)

In [8]:
Keyword = '전지적 독자 시점'
twint_loop(Keyword, '2020-01-01', '2020-04-25')

Directory 전지적독자시점 already exists
Getting 2020-01-01 
Problem with 2020-01-01.
Getting 2020-01-02 
Problem with 2020-01-02.
Getting 2020-01-03 
Problem with 2020-01-03.
Getting 2020-01-04 
Problem with 2020-01-04.
Getting 2020-01-05 
Problem with 2020-01-05.
Getting 2020-01-06 
Problem with 2020-01-06.
Getting 2020-01-07 
Problem with 2020-01-07.
Getting 2020-01-08 
Problem with 2020-01-08.
Getting 2020-01-09 
Problem with 2020-01-09.
Getting 2020-01-10 
Problem with 2020-01-10.
Getting 2020-01-11 
Problem with 2020-01-11.
Getting 2020-01-12 
Problem with 2020-01-12.
Getting 2020-01-13 
Problem with 2020-01-13.
Getting 2020-01-14 
Problem with 2020-01-14.
Getting 2020-01-15 
Problem with 2020-01-15.
Getting 2020-01-16 
Problem with 2020-01-16.
Getting 2020-01-17 
Problem with 2020-01-17.
Getting 2020-01-18 
Problem with 2020-01-18.
Getting 2020-01-19 
Problem with 2020-01-19.
Getting 2020-01-20 
Problem with 2020-01-20.
Getting 2020-01-21 
Problem with 2020-01-21.
Getting 2020-01-22 
Pr